In [10]:
# define project directory

project_dir = '/home/connor/repositories/hoim'

In [11]:
from utils import load_df, remove_duplicated, best_group
import seaborn as sb
sb.set_color_codes("muted")

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from os.path import join

In [12]:
# Results dir
results_dir = join(project_dir, 'results')

# dict for all results
dfs_all = {}

In [13]:
sat_dirs = [
            'sat_3rd',
            # 'sat_2nd',
]
sat_name = {sd: n for sd, n in zip(sat_dirs, ['3rd', '2nd'])}
groupbys = [

    ['rho', 'd', 'coupling_const', 'shil', 'num_cycles'],
            ['min_gap', 'rho', 'd', 'coupling_const', 'shil','num_cycles'],
]
cols = ['coupling_const', 'shil', 'rho', 'd', 'min_gap', 'csp_size', 'normalize', 'valid', 'noise_std',
        'times', 'step', 'num_steps', 'seed', 'energy', 'sample', 'problem', 'num_constraints', 'num_cycles']

def load(sat_dir, groupby):
    data_dir = join(results_dir, sat_dir)
    df = load_df(data_dir, max_entries=None, cols=cols)
    print(df.shape)
    df = remove_duplicated(df, [c for c in cols if c in df.columns], True)
    if 'min_gap' not in df.columns:
        df['min_gap'] = 1
    df['model'] = sat_dir
    grouped = df[df['step'] == df['num_steps'] - 1].groupby(['csp_size'] + groupbys[-1] + ['model'])
    
    if 'min_gap' in df.columns:
        df = df.query('min_gap in [1, 5, 10, 13]')
    df['energy_fmin'] = 100 * (1 - (df['energy'] / df['num_constraints']))

    return df

for sat_dir, groupby in zip(sat_dirs, groupbys):
    
    df = load(sat_dir, groupby)
    dfs_all[sat_dir] = df
    
    print(sat_dir, dfs_all[sat_dir].shape)
    sort_by = ['csp_size', 'energy']
    grouped = df[df['step'] == df['num_steps'] - 1].groupby(['csp_size'] + groupby)
    print(grouped[['energy', 'valid']].mean().sort_values(by=sort_by).to_string())
 
    

  0%|          | 0/2 [00:00<?, ?it/s]

(8192, 16)
0 0
sat_3rd (8192, 19)
                                                 energy  valid
csp_size rho d  coupling_const shil num_cycles                
20       1   -1 1              1    256         0.53125    0.5
